In [1]:
from dotenv import load_dotenv
import os

load_dotenv()

True

In [2]:
GROQ_API_KEY = os.getenv("GROQ_API_KEY")

In [3]:
from langchain.agents.middleware import HumanInTheLoopMiddleware
from langgraph.checkpoint.memory import InMemorySaver
from langchain_groq import ChatGroq
from langchain.agents import create_agent
from langchain.tools import tool

In [10]:
@tool("create_file")
def create_text_file(filename: str, content: str) -> dict:
    """
    Tạo một file .txt và ghi nội dung vào file.
    Luôn trả về JSON để LLM dễ parse.
    """
    import os
    import traceback

    try:
        # Tự thêm .txt nếu thiếu
        if not filename.endswith(".txt"):
            filename += ".txt"

        # Nếu user truyền path absolute như F:/result.txt
        # thì filename có thể là 'F:/result.txt'
        # Nếu chỉ truyền tên file -> ghi vào thư mục hiện tại
        file_path = os.path.abspath(filename)

        # Tự tạo thư mục nếu chưa có
        os.makedirs(os.path.dirname(file_path), exist_ok=True)

        # Ghi file
        with open(file_path, "w", encoding="utf-8") as f:
            f.write(str(content))

        return {
            "success": True,
            "filename": filename,
            "absolute_path": file_path,
            "message": "File created successfully"
        }

    except Exception as e:
        return {
            "success": False,
            "filename": filename,
            "error": str(e),
            "trace": traceback.format_exc()
        }
    
@tool("delete_file")
def delete_text_file(filename: str) -> dict:
    """
    Xóa một file .txt hoặc bất kỳ file nào theo tên/path.
    Luôn trả về JSON để LLM dễ parse.
    """
    import os
    import traceback

    try:
        # Nếu thiếu .txt thì tự thêm — giống tool create_file
        if not filename.endswith(".txt"):
            filename += ".txt"

        file_path = os.path.abspath(filename)

        # Kiểm tra file tồn tại
        if not os.path.exists(file_path):
            return {
                "success": False,
                "filename": filename,
                "absolute_path": file_path,
                "message": "File does not exist"
            }

        # Tiến hành xóa file
        os.remove(file_path)

        return {
            "success": True,
            "filename": filename,
            "absolute_path": file_path,
            "message": "File deleted successfully"
        }

    except Exception as e:
        return {
            "success": False,
            "filename": filename,
            "absolute_path": file_path,
            "error": str(e),
            "trace": traceback.format_exc()
        }


In [ ]:
human_approval = HumanInTheLoopMiddleware(
    interrupt_on={
        "create_file": {"allowed_decisions": ["approve", "reject"]},
        "delete_file": {"allowed_decisions": ["approve", "reject"]}
    },
    description_prefix="Tool execution pending approval",
)

In [6]:
llm = ChatGroq(model="qwen/qwen3-32b", api_key=GROQ_API_KEY)

agent = create_agent(
    model=llm,
    tools=[create_text_file],
    middleware=[human_approval],
    checkpointer=InMemorySaver()
)

In [7]:
from langgraph.types import Command

In [9]:
config = {"configurable": {"thread_id": "1"}}

result = agent.invoke(
    {
        "messages": [
            {
                "role": "user",
                "content": "Can you create file named test4.txt with the content is Nam đẹp trai located at F:/"
            }
        ]
    },
    config=config
)
print(result["messages"][-1].content)

result = agent.invoke(
    Command(
        resume={"decisions": [{"type": "approve"}]}
    ),
    config=config
)
print(result["messages"][-1].content)


File "test4.txt" đã được tạo thành công tại đường dẫn `F:/test4.txt` với nội dung "Nam đẹp trai". Bạn có thể kiểm tra file này trong thư mục F:/. Cần hỗ trợ thêm gì không? 😊
